### nltk 전처리 확인해 볼 것
### sklearn ConterVectorizer 쉽게 만들 수 있다.

In [1]:
#nltk에 있는 movie_reviews 가지고 오기!
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/jun/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [2]:
from nltk.corpus import movie_reviews

- 자연어 처리할 때
- 카운팅 기반으로 잡을 수 있도
- 빈도 기반으로도 잡을 수 있다.

- 카운팅 기반으로 잡게되면 결국 내가 원하는 형태소를 얼마나 전체 리뷰에서 가지고 있는가? - BOW Bag of words 카운팅 기반
- 빈도의 개념으로 들어가면 단순하게 10 5 6 카운트에서 -> 빈도가 되니까 10/21 5/21 6/21 -> TF-IDF 

- movie_reviws 주요 함수
- fileids() - 영화리뷰 문서들의 id를 반환, id만 가지고 올 수 있음
- categoris() - 리뷰 문서들에 대한 분류, 라벨을 보여준다. 긍정 또는 부정
- raw() - 리뷰 문서의 문자열 리스트 반환
- sents() - 리뷰 문서 원문에 대한 NLTK 토큰화 진행 ( sent_tokenize)
- words() - 토큰화 결과 반환

In [3]:
print(movie_reviews.fileids()[:5])

['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt']


In [15]:
fl1 =movie_reviews.fileids()[1]

In [16]:
movie_reviews.raw(fl1)

'the happy bastard\'s quick movie review \ndamn that y2k bug . \nit\'s got a head start in this movie starring jamie lee curtis and another baldwin brother ( william this time ) in a story regarding a crew of a tugboat that comes across a deserted russian tech ship that has a strangeness to it when they kick the power back on . \nlittle do they know the power within . . . \ngoing for the gore and bringing on a few action sequences here and there , virus still feels very empty , like a movie going for all flash and no substance . \nwe don\'t know why the crew was really out in the middle of nowhere , we don\'t know the origin of what took over the ship ( just that a big pink flashy thing hit the mir ) , and , of course , we don\'t know why donald sutherland is stumbling around drunkenly throughout . \nhere , it\'s just " hey , let\'s chase these people around with some robots " . \nthe acting is below average , even from the likes of curtis . \nyou\'re more likely to get a kick out of h

In [17]:
#토크나이즈 진행 
print(movie_reviews.sents(fl1)[:5])

[['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', 'review', 'damn', 'that', 'y2k', 'bug', '.'], ['it', "'", 's', 'got', 'a', 'head', 'start', 'in', 'this', 'movie', 'starring', 'jamie', 'lee', 'curtis', 'and', 'another', 'baldwin', 'brother', '(', 'william', 'this', 'time', ')', 'in', 'a', 'story', 'regarding', 'a', 'crew', 'of', 'a', 'tugboat', 'that', 'comes', 'across', 'a', 'deserted', 'russian', 'tech', 'ship', 'that', 'has', 'a', 'strangeness', 'to', 'it', 'when', 'they', 'kick', 'the', 'power', 'back', 'on', '.'], ['little', 'do', 'they', 'know', 'the', 'power', 'within', '.'], ['.'], ['.']]


In [10]:
#토큰 반환을 진행한다.
print(movie_reviews.words(fl1)[:10])

['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', 'review', 'damn', 'that']


In [18]:
fileid = movie_reviews.fileids()[0]

'neg/cv000_29416.txt'

In [28]:
## 불용오 제거하기 
## NLTK 불용어 사전
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords # 일반적으로 분석대상이 아닌 단어들 

tokenizer = RegexpTokenizer("['\w']{3,}") # 정규표현식 토크나이저 
english_stops = set(stopwords.words('english'))  #영어 불용어를 가져옴

#words() 대신 raw()로 원문을 가져옴
documents = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]

#stopwords의 적용과 토큰화를 동시에 수행
tokens =[[token for token in tokenizer.tokenize(str(doc)) if token not in english_stops] for doc in documents]
word_count = {}
for text in tokens:
    for word in text:
        word_count[word] = word_count.get(word,0) +1

sorted_features = sorted(word_count, key=word_count.get, reverse=True)

print('num of features:', len(sorted_features))
for word in sorted_features[:100]:
    print(f"count of'{word}':{word_count[word]}", end=', ')

num of features: 43030
count of'film':8935, count of'one':5791, count of'movie':5538, count of'like':3690, count of'even':2564, count of'time':2409, count of'good':2407, count of'story':2136, count of'would':2084, count of'much':2049, count of'also':1967, count of'get':1945, count of'character':1928, count of'two':1911, count of'well':1905, count of'first':1834, count of'characters':1817, count of'see':1746, count of'way':1693, count of'make':1641, count of'life':1571, count of'really':1558, count of'films':1528, count of'plot':1502, count of'little':1500, count of'people':1428, count of'could':1400, count of'bad':1394, count of'scene':1394, count of'never':1372, count of'best':1331, count of'man':1312, count of'new':1290, count of'scenes':1274, count of'many':1268, count of'know':1214, count of'movies':1202, count of'director':1181, count of'action':1171, count of'he's':1154, count of'great':1147, count of'another':1113, count of'love':1111, count of'made':1084, count of'big':1063, co

### 빈도가 높은 상위 단어만 추출해 보자

In [32]:
word_count

{'plot': 1502,
 'two': 1911,
 'teen': 150,
 'couples': 25,
 'church': 68,
 'party': 182,
 'drink': 32,
 'drive': 105,
 'get': 1945,
 'accident': 103,
 'one': 5791,
 'guys': 267,
 'dies': 104,
 'girlfriend': 211,
 'continues': 88,
 'see': 1746,
 'life': 1571,
 'nightmares': 26,
 "what's": 232,
 'deal': 219,
 'watch': 603,
 'movie': 5538,
 'sorta': 10,
 'find': 782,
 'critique': 61,
 'mind': 448,
 'fuck': 16,
 'generation': 95,
 'touches': 55,
 'cool': 207,
 'idea': 386,
 'presents': 78,
 'bad': 1394,
 'package': 30,
 'makes': 992,
 'review': 294,
 'even': 2564,
 'harder': 33,
 'write': 119,
 'since': 768,
 'generally': 103,
 'applaud': 10,
 'films': 1528,
 'attempt': 263,
 'break': 175,
 'mold': 14,
 'mess': 159,
 'head': 385,
 'lost': 407,
 'highway': 28,
 'memento': 10,
 'good': 2407,
 'ways': 189,
 'making': 602,
 'types': 48,
 'folks': 73,
 'snag': 2,
 'correctly': 17,
 'seem': 574,
 'taken': 224,
 'pretty': 528,
 'neat': 32,
 'concept': 114,
 'executed': 46,
 'terribly': 58,
 'prob

In [29]:
word_features = sorted_features[:500]

In [31]:
word_features

['film',
 'one',
 'movie',
 'like',
 'even',
 'time',
 'good',
 'story',
 'would',
 'much',
 'also',
 'get',
 'character',
 'two',
 'well',
 'first',
 'characters',
 'see',
 'way',
 'make',
 'life',
 'really',
 'films',
 'plot',
 'little',
 'people',
 'could',
 'bad',
 'scene',
 'never',
 'best',
 'man',
 'new',
 'scenes',
 'many',
 'know',
 'movies',
 'director',
 'action',
 "he's",
 'great',
 'another',
 'love',
 'made',
 'big',
 'end',
 'back',
 'something',
 'still',
 'seems',
 'work',
 'makes',
 "there's",
 'world',
 'however',
 'every',
 'though',
 'better',
 'real',
 'seen',
 'enough',
 'take',
 'around',
 'going',
 'performance',
 'audience',
 'role',
 'old',
 'gets',
 'may',
 'things',
 'think',
 'years',
 'last',
 'actually',
 'comedy',
 'look',
 'funny',
 'long',
 'almost',
 'thing',
 'fact',
 'nothing',
 'say',
 'right',
 'although',
 'played',
 'find',
 "that's",
 'john',
 'come',
 'ever',
 'since',
 'cast',
 'script',
 'year',
 'plays',
 'star',
 'young',
 'comes',
 'show

### 카운팅을 하기 위해서 빈도에 대해서 확인해 보자!

In [33]:
##주어진 document를 feature로 변환하는 함수, word_features를 사용
def document_features(document, word_features):
    word_count = {}
    for word in document: #document에 있는 단어들에 대해 빈도수를 계산 
        word_count[word] = word_count.get(word,0)+1
        
    features = []
    # word_features의 단어에 대해 계산된 빈도수를 feature에 추가 
    for word in word_features:
        features.append(word_count.get(word,0))#빈도가 없는 단어는 0을 입력
    return features

In [37]:
meta_features = ['문법기초반','문법응용반','분석기초반','분석중급반(상)','분석고급반','R문법기초반','SQL기초반']
ex_doc = ['문법기초반','문법응용반','분석중급반(하)','태블로시각화반','SQL기초반','SQL중급반']

print(document_features(meta_features, ex_doc))

[1, 1, 0, 0, 1, 0]


- 내가 원하는 내가 보고자하는 특정 단어나 워드를 기반으로 카운팅을 진행하고 빈도가 없는 경우는 0으로 출력을 한다.

- 0 희소행렬 sparse 

### 사이킷런으로 간단하게 만들 수 있다.

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

In [40]:
reviews = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]


## countverizer 함수
- tokenizer 함수형태로 토크나이저 
- stop_words 불용어로 진행한다. english 영어인 경우는
- ngram_range( ngram 범위 지정 )
- max_df 가장 많은 빈도
- min_df 가장 작은 빈도
- max_features
- vocabulary 특성으로 사용할 단어 저장
- binary True값을 주면 빈도 대신 1,0 특성값을 생성할 수 있다.

---
### 주요 메서드
- fit(데이터) 집어 넣어서 메서드를 사용한다는 뜻
- transform() - fit에서 생성한 집합을 다시 변환한다-> 카운팅
- fit_transform() fit, transform 동시에 진행
- get_feature_names_out() - 특성에 대한 이름, 즉 단어를 순서대로 반환한다. 

In [42]:
cv = CountVectorizer() #기본값

# 기존에 만들었던 word 특성을 집어넣어서 지정할 수 있다. word_features
cv1 =CountVectorizer(vocabulary =word_features )

# 특성을 넣지 않는 경우 빈도 기반으로해서 추출해줘
cv2 = CountVectorizer(max_features=500)

In [46]:
rv_cv =cv.fit_transform(reviews)

rv_cv1= cv1.fit_transform(reviews)

rv_cv2= cv2.fit_transform(reviews)


In [47]:
rv_cv

<2000x39659 sparse matrix of type '<class 'numpy.int64'>'
	with 666842 stored elements in Compressed Sparse Row format>

In [50]:
rv_cv1

<2000x500 sparse matrix of type '<class 'numpy.int64'>'
	with 186654 stored elements in Compressed Sparse Row format>

In [49]:
rv_cv2

<2000x500 sparse matrix of type '<class 'numpy.int64'>'
	with 296902 stored elements in Compressed Sparse Row format>

In [70]:
print(cv.get_feature_names_out()[:20])
print(cv1.get_feature_names_out()[:20])
print(cv2.get_feature_names_out()[:20])

['00' '000' '0009f' '007' '00s' '03' '04' '05' '05425' '10' '100' '1000'
 '10000' '100m' '101' '102' '103' '104' '105' '106']
['film' 'one' 'movie' 'like' 'even' 'time' 'good' 'story' 'would' 'much'
 'also' 'get' 'character' 'two' 'well' 'first' 'characters' 'see' 'way'
 'make']
['10' 'able' 'about' 'act' 'acting' 'action' 'actor' 'actors' 'actually'
 'after' 'again' 'against' 'alien' 'all' 'almost' 'along' 'already' 'also'
 'although' 'always']


In [54]:
rv_cv1.shape

(2000, 500)

In [55]:
rv_cv.shape

(2000, 39659)

In [56]:
rv_cv2.shape

(2000, 500)

In [64]:
print(rv_cv[0, :30])

  (0, 9)	10


In [66]:
print(rv_cv1[0, :30])

  (0, 0)	6
  (0, 1)	3
  (0, 2)	6
  (0, 3)	3
  (0, 4)	3
  (0, 6)	2
  (0, 8)	1
  (0, 10)	1
  (0, 11)	3
  (0, 12)	2
  (0, 13)	2
  (0, 14)	1
  (0, 16)	1
  (0, 17)	2
  (0, 18)	3
  (0, 19)	5
  (0, 20)	1
  (0, 21)	2
  (0, 22)	2
  (0, 23)	1
  (0, 24)	2
  (0, 25)	1
  (0, 27)	2


In [65]:
print(rv_cv2[0, :30])

  (0, 22)	20
  (0, 21)	3
  (0, 27)	13
  (0, 13)	6
  (0, 29)	1
  (0, 10)	2
  (0, 9)	2
  (0, 8)	2
  (0, 19)	1
  (0, 2)	2
  (0, 7)	1
  (0, 18)	1
  (0, 20)	1
  (0, 17)	1
  (0, 0)	10


### 필수과제
- 영화리뷰데이터 네이버 한국어 데이터
- 한국어 리뷰를 전처리 진행하고 -> countverctorizer을 사용해서 전처리를 진행해 주세요.
- 문자열 자체 전처를 통해서 행렬 만들어줘야 한다.
- 이걸 가지고 학습을 할 수 있는 것이고- >실제 x축과 y축에 어떤 것이 들어가서 카운팅이 되는지를 정확히 이해해야 한다.

- 기본 cv값과
- 본인이 전처리한 cv값과
- 빈도기반 cv값을 

- 3가지로 나눠서 카운팅 비교 분석을 진행해 주세요!